# Pipeline 1

In [1]:
from pipeline import Falcon2Linker, SerialAnnotator, T5Converter
import time
from pprint import pprint

In [2]:
linker = Falcon2Linker()
annotator = SerialAnnotator()
converter = T5Converter()

In [3]:
utterance = "Who is the wife of Barack Obama"

In [4]:
linked = linker.link(utterance)
# rules = [1,2,3,4,5,8,9,10,12,13,14]
# linked = process_text_E_R(utterance, rules)
pprint("Linked")
pprint(linked)

pprint("Annotated")
annotated = annotator.annotate(**linked)
pprint(annotated)

pprint("Converted")
converted = converter.preprocess_inputs(**annotated)
pprint(converted)

recieved utterance
0 : ['Who is the wife of Barack Obama']


[reRank_relations]: who is the wife of Barack Obama 
SPARQL Requests Made: 1 
SPARQL Requests Total Time: 0.7111196517944336 
SPARQL Avg Time per Qn: 0.7111196517944336
['Who is the wife of Barack Obama', [['<http://www.wikidata.org/entity/P26>', 'wife'], ['<http://www.wikidata.org/entity/P2848>', 'wife'], ['<http://www.wikidata.org/entity/P140>', 'wife'], ['<http://www.wikidata.org/entity/P451>', 'wife'], ['<http://www.wikidata.org/entity/P600>', 'wife']], [['<http://www.wikidata.org/entity/Q76>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q649593>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q4808526>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q4858106>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q643049>', 'Barack obama']], 0, 0, 0, 0]
'Linked'
{'ents': [{'id': 'Q76',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org/entity/Q76'},
          {'id': 'Q649593',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org

{'fragments': ['[DEF]',
               'wd:',
               'Q76 Barack Obama',
               '[DEF]',
               'wd:',
               'Q649593 Barack Obama Sr.',
               '[DEF]',
               'wd:',
               'Q4808526 assassination threats against Barack Obama',
               '[DEF]',
               'wd:',
               'Q4858106 Barack Obama Academy',
               '[DEF]',
               'wd:',
               'Q643049 family of Barack Obama',
               '[DEF]',
               'wdt:',
               'P26 spouse',
               '[DEF]',
               'wdt:',
               'P2848 Wi-Fi access',
               '[DEF]',
               'wdt:',
               'P140 religion',
               '[DEF]',
               'wdt:',
               'P451 unmarried partner',
               '[DEF]',
               'wdt:',
               'P600 Wine AppDB ID'],
 'utterance': 'Who is the wife of Barack Obama'}
'Converted'
('Who is the wife of Barack Obama <extra_id_59> <ext

In [5]:
def pipe(utterance, wikisparql):
    linked = linker.link(utterance)
    annotated = annotator.annotate(**linked)
    converted = converter.preprocess(**annotated, wikisparql=wikisparql)
    return linked, annotated, converted

In [6]:
def pipe_batch(utterances, wikisparqls):
  batched = []
  link_batch_start = time.time()
  linked = linker.link_batch(utterances)
  link_batch_end = time.time()
  annotator_time = 0
  converter_time = 0
  for i, single_linked in enumerate(linked):
    s = time.time()
    annotated = annotator.annotate(**single_linked)
    e = time.time()
    try:
      annotator_time += e - s
    except Exception as e:
      print("[Annotator Error]:", e)
      continue
    s = time.time()
    try:
      converted = converter.preprocess(**annotated, wikisparql=wikisparqls[i])
    except Exception as e:
      print("[Converter Error]:", e)
      continue
      
    e = time.time()
    converter_time += e - s
    batched.append([single_linked, annotated, converted])
  print("Link batch time:", link_batch_end - link_batch_start)
  print("Anno batch time:", annotator_time)
  print("Conv batch time:", converter_time)
  return batched

## Data

In [7]:
from pathlib import Path
import pandas as pd
import json

data_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "train.json"

In [8]:
from pprint import pprint
with open(data_path) as f:
  data_dict = json.load(f)

count = 0

for item in data_dict:
  paraphrased_question = item["paraphrased_question"]
  question = item["question"]
  if paraphrased_question == None:
    raise Exception
  if question == None:
    count += 1
    # print(len(paraphrased_question) > 2)
    # pprint(item)
  if item["sparql_wikidata"] == None:
    raise Exception
  
  item["new_question"] = paraphrased_question if len(paraphrased_question) > 2 and paraphrased_question != [] else question

print("Missing question count:", count)

df = pd.DataFrame.from_dict(data_dict)
df = df.dropna(axis=0, subset=['new_question'])
n_entries = len(df)
print("Number of rows:", n_entries)

Missing question count: 56
Number of rows: 24125


In [9]:
assert(len(df[df.sparql_wikidata.isna()]) == 0)

In [10]:
responses = []
batch_qns = []
batch_ans = []
truncated_data = data_dict[6000:]
total_len = len(truncated_data)
last = 0
for i, data in enumerate(truncated_data):
    print("Pipeline iter", i)
    question = data["new_question"]
    answer = data["sparql_wikidata"]
    # try
    #     linked, annotated, converted = pipe(question, answer)
    # except Exception as e:
    #     print(e)
    #     continue
    # responses.append([linked, annotated, converted])
    if (question and answer) and len(question) >= 2:
        batch_qns.append(question)
        batch_ans.append(answer)
    if ((i + 1) % 50) == 0 or i == total_len - 1:
        print("[Pipeline1]:", f"Linking {last}-{i}")
        try:
            for linked, annotated, converted in pipe_batch(batch_qns, batch_ans):
                responses.append([linked, annotated, converted])
            batch_qns = []
            batch_ans = []
            last = i
        except Exception as e:
            raise e
            # print(e)
            # continue
        with open(f"../t5-for-sparql/falcon_links/4/link_{i}.json", "w") as f:
            json.dump(responses, f, indent=2, separators=(',',':'))

Pipeline iter 0
Pipeline iter 1
Pipeline iter 2
Pipeline iter 3
Pipeline iter 4
Pipeline iter 5
Pipeline iter 6
Pipeline iter 7
Pipeline iter 8
Pipeline iter 9
Pipeline iter 10
Pipeline iter 11
Pipeline iter 12
Pipeline iter 13
Pipeline iter 14
Pipeline iter 15
Pipeline iter 16
Pipeline iter 17
Pipeline iter 18
Pipeline iter 19
Pipeline iter 20
Pipeline iter 21
Pipeline iter 22
Pipeline iter 23
Pipeline iter 24
Pipeline iter 25
Pipeline iter 26
Pipeline iter 27
Pipeline iter 28
Pipeline iter 29
Pipeline iter 30
Pipeline iter 31
Pipeline iter 32
Pipeline iter 33
Pipeline iter 34
Pipeline iter 35
Pipeline iter 36
Pipeline iter 37
Pipeline iter 38
Pipeline iter 39
Pipeline iter 40
Pipeline iter 41
Pipeline iter 42
Pipeline iter 43
Pipeline iter 44
Pipeline iter 45
Pipeline iter 46
Pipeline iter 47
Pipeline iter 48
Pipeline iter 49
[Pipeline1]: Linking 0-49
1 : ['What is the number of Presentations Dick Clark Gave?']
1 : ['Name the method that begins with W.']
1 : ['What is endemic to kang

11 : ['Are Wycliffes Bible and The LDS edition versions of the Bible?']
 : ['who pharmaceutical item for has dynamic fixing of amoxicillin?']
1 : ['WHICH IS THE ACADEMIC DEGREE AND ACADEMIC MAJOR OF BASHAR ALASSAD HAS EDUCATED IN DAMASCUS UNIVERSITY']
1 : ['What work environment compiled Historia Coelestis Britannica?']
1 : ['At the 61st Primetime Emmy Awards, did Chevy Chase receive an Emmy?']
1 : ['Give me the names of the pupils of this Fur Elise songwriter.']


2 : ['What is a religious order of knight ?']


3 : ['Let Me know CEREMONY WHICH Begin WITH THE LETTER U.']


4 : ['The second-level administrative country subdivision is in which part of Adelie Land?']


...

[reRank_relations]: wHICH IS THE ORGANIZATION OFANTHEM THAT IS ANTHEM OF EUROPE AND WHICH STARTS WITH C 
SPARQL Requests Made: 484 
SPARQL Requests Total Time: 351.65407276153564 
SPARQL Avg Time per Qn: 0.7265580015734208


[reRank_relations]: Wh auden and the Pulitzer Prize for Poetry A transplant from Great Britain wins the hearts and minds of Pulitzer juries Wystan Hugh Auden, the British poet, moved to the United States in 1939 He taught at the University of Michigan and Swarthmore College during World War II 
SPARQL Requests Made: 840 
SPARQL Requests Total Time: 611.5378284454346 
SPARQL Avg Time per Qn: 0.7280212243398031


[reRank_relations]: state the number of matches Roberto Baggio played for Bologna FC 1909 and the number of goals he scored 
SPARQL Requests Made: 440 
SPARQL Requests Total Time: 320.4607846736908 
SPARQL Avg Time per Qn: 0.7283199651674791


[reRank_relations]: Wh auden and the Pulitzer Prize for Poetry A transplant from Great Britain wins the hearts and minds of Pulitzer juries Wystan Hugh Auden, the British poet, moved to the United States in 1939 He taught at the University of Michigan and Swarthmore College during World War II 
SPARQL Requests Made: 80 
SPARQL Requests Total Time: 58.53369331359863 
SPARQL Avg Time per Qn: 0.7316711664199829


[reRank_relations]: wHICH IS THE ORGANIZATION OFANTHEM THAT IS ANTHEM OF EUROPE AND WHICH STARTS WITH C 
SPARQL Requests Made: 782 
SPARQL Requests Total Time: 571.1754288673401 
SPARQL Avg Time per Qn: 0.7304033617229413


In [ ]:
# qns = ['What is the operating income for Qantas?', 'What is Mary Lou Rettons International Olympic Committee athlete ID.']
# l = linker.link_batch(qns)
# print(l)

In [ ]:
with open("home.json", "w") as f:
  json.dump(responses, f, indent=2 ,separators=(',', ': '))